In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scipy

plt.style.use('rho')

c1, c2, c3, c4, c5, c6, *_cs = [x['color'] for x in list(plt.rcParams['axes.prop_cycle'])]

In [2]:
import requests

In [3]:
BASE_URL = "https://en.wikipedia.org/w/api.php"

base_params = {
    'action': 'query',
    'utf8': 1,
    'maxlag': 1,
    'format': 'json'
}
params = {
    'list': 'geosearch',
    'gscoord': '41.3129|-72.93',
    'gsradius': 100,
    'gslimit': 200,
}

headers = {
    'User-Agent': 'Loca v0.1 (https://github.com/loca-ai/loca-vision)'
}

r = requests.get(BASE_URL, params={**base_params, **params}, headers=headers)
r.raise_for_status()
obj = r.json()
obj

{'batchcomplete': '',
 'query': {'geosearch': [{'pageid': 1300586,
    'ns': 0,
    'title': 'Morse College',
    'lat': 41.312583333333336,
    'lon': -72.9305,
    'dist': 54.6,
    'primary': ''},
   {'pageid': 1300584,
    'ns': 0,
    'title': 'Ezra Stiles College',
    'lat': 41.3125,
    'lon': -72.9306,
    'dist': 67,
    'primary': ''},
   {'pageid': 2223802,
    'ns': 0,
    'title': 'Payne Whitney Gymnasium',
    'lat': 41.31361111111111,
    'lon': -72.93055555555556,
    'dist': 91.7,
    'primary': ''}]}}

In [7]:
from typing import Sequence
def geosearch_pages(lat: float, lon: float, radius: float, limit=200) -> Sequence[int]:
    """Generates all pages with location no more than the given radius (in meters)
    away from the given coordinates. """
    params = {
        "list": "geosearch",
        "gscoord": f"{lat}|{lon}",
        "gsradius": radius,
        "gslimit": limit,
    }

    r = requests.get(BASE_URL, params={**base_params, **params}, headers=headers)
    r.raise_for_status()
    json = r.json()
    return [page["pageid"] for page in json["query"]["geosearch"]]

pages = geosearch_pages(41.312, -72.92, 200)
pages

[26979629, 3148044, 13574689]

In [6]:
from loca_vision.wiki_parser import search_monuments_nearby
from loca_vision.coord import Coord
search_monuments_nearby(Coord(42.31142, -71.08763), 20, 10)

KeyError: 'query'

In [157]:
def get_images_text(pageids):
    params = {
        'prop': 'images|extracts|pageimages',
        'formatversion': 2,        
        'pageids': '|'.join(map(str, pageids)),
        'redirects': 1
    }
    print({**base_params, **params})
    r = requests.get(BASE_URL, params={**base_params, **params}, headers=headers)
    r.raise_for_status()
    return r.json()

page = get_images_text([x['pageid'] for x in obj['query']['geosearch']])
page

{'action': 'query', 'utf8': 1, 'maxlag': 1, 'format': 'json', 'prop': 'images|extracts|pageimages', 'formatversion': 2, 'pageids': '1300586|1300584|2223802', 'redirects': 1}


{'continue': {'imcontinue': '1300586|Morse_College_courtyard.JPG',
  'excontinue': 1,
  'continue': '||pageimages'},
 'warnings': {'extracts': {'warnings': '"exlimit" was too large for a whole article extracts request, lowered to 1.\nHTML may be malformed and/or unbalanced and may omit inline images. Use at your own risk. Known problems are listed at https://www.mediawiki.org/wiki/Special:MyLanguage/Extension:TextExtracts#Caveats.'}},
 'query': {'pages': [{'pageid': 1300584,
    'ns': 0,
    'title': 'Ezra Stiles College',
    'images': [{'ns': 6, 'title': 'File:Commons-logo.svg'},
     {'ns': 6, 'title': 'File:EzraStilesshield.png'},
     {'ns': 6, 'title': 'File:Ezra Stiles College Courtyard.jpg'},
     {'ns': 6, 'title': 'File:OOjs UI icon edit-ltr-progressive.svg'},
     {'ns': 6, 'title': 'File:Question book-new.svg'},
     {'ns': 6, 'title': 'File:Symbol category class.svg'},
     {'ns': 6, 'title': 'File:Wikinews-logo.svg'},
     {'ns': 6, 'title': 'File:Yale Harkness Tower.JPG'

In [50]:
def get_image_urls(imgnames):
    params = {
        'prop': 'imageinfo',    
        'titles': '|'.join(map(str, imgnames)),
        'iiprop': 'url'
    }
    print({**base_params, **params})
    r = requests.get(BASE_URL, params={**base_params, **params}, headers=headers)
    r.raise_for_status()
    return r.json()

urls = get_image_urls(im['title'] for im in page['query']['pages'][0]['images'])
urls

{'action': 'query', 'utf8': 1, 'maxlag': 1, 'format': 'json', 'prop': 'imageinfo', 'titles': 'File:Commons-logo.svg|File:EzraStilesshield.png|File:Ezra Stiles College Courtyard.jpg|File:OOjs UI icon edit-ltr-progressive.svg|File:Question book-new.svg|File:Symbol category class.svg|File:Wikinews-logo.svg|File:Yale Harkness Tower.JPG', 'iiprop': 'url'}


{'batchcomplete': '',
 'query': {'pages': {'-1': {'ns': 6,
    'title': 'File:Ezra Stiles College Courtyard.jpg',
    'missing': '',
    'known': '',
    'imagerepository': 'shared',
    'imageinfo': [{'url': 'https://upload.wikimedia.org/wikipedia/commons/e/ed/Ezra_Stiles_College_Courtyard.jpg',
      'descriptionurl': 'https://commons.wikimedia.org/wiki/File:Ezra_Stiles_College_Courtyard.jpg',
      'descriptionshorturl': 'https://commons.wikimedia.org/w/index.php?curid=37475437'}]},
   '-2': {'ns': 6,
    'title': 'File:Wikinews-logo.svg',
    'missing': '',
    'known': '',
    'imagerepository': 'shared',
    'imageinfo': [{'url': 'https://upload.wikimedia.org/wikipedia/commons/2/24/Wikinews-logo.svg',
      'descriptionurl': 'https://commons.wikimedia.org/wiki/File:Wikinews-logo.svg',
      'descriptionshorturl': 'https://commons.wikimedia.org/w/index.php?curid=1023995'}]},
   '-3': {'ns': 6,
    'title': 'File:Yale Harkness Tower.JPG',
    'missing': '',
    'known': '',
    'im

In [12]:
imgsearch = 'http://www.emporis.com/images/show/767485-Large-fullheightview-view-from-the-southwest.jpg'
r = requests.get('https://upload.wikimedia.org/wikipedia/en/b/b7/EzraStilesshield.png', headers=headers)
type(r.content)

bytes

In [81]:
from google.cloud import vision

def create_product_set(
        project_id, location, product_set_id, product_set_display_name):
    """Create a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_set_display_name: Display name of the product set.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Create a product set with the product set specification in the region.
    product_set = vision.ProductSet(
            display_name=product_set_display_name)

    # The response is the product set with `name` populated.
    response = client.create_product_set(
        parent=location_path,
        product_set=product_set,
        product_set_id=product_set_id)

    # Display the product set information.
    print('Product set name: {}'.format(response.name))

create_product_set('loca-346705', 'us-east1', 'monuments', 'Monuments')

Product set name: projects/loca-346705/locations/us-east1/productSets/monuments


In [115]:
from google.cloud import vision
from google.protobuf import field_mask_pb2 as field_mask

def create_product(
        project_id, location, product_id, product_display_name,
        product_category):
    """Create one product.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
        product_display_name: Display name of the product.
        product_category: Category of the product.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Create a product with the product specification in the region.
    # Set product display name and product category.
    product = vision.Product(
        display_name=product_display_name,
        product_category=product_category)

    # The response is the product with the `name` field populated.
    response = client.create_product(
        parent=location_path,
        product=product,
        product_id=product_id)

    # Display the product information.
    print('Product name: {}'.format(response.name))

# create_product('loca-346705', 'us-east1', 'lipstick-tank', 'Lipstick (Ascending) on Caterpillar Tracks', 'general-v1')
# create_product('loca-346705', 'us-east1', 'morse-college', 'Morse College', 'general-v1')

Product name: projects/loca-346705/locations/us-east1/products/morse-college


In [132]:
from google.cloud import vision

def add_product_to_product_set(
        project_id, location, product_id, product_set_id):
    """Add a product to a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # Get the full path of the product.
    product_path = client.product_path(
        project=project_id, location=location, product=product_id)

    # Add the product to the product set.
    client.add_product_to_product_set(
        name=product_set_path, product=product_path)
    print('Product added to product set.')

# add_product_to_product_set('loca-346705', 'us-east1', 'lipstick-tank', 'monuments')
# add_product_to_product_set('loca-346705', 'us-east1', 'morse-college', 'monuments')

Product added to product set.
Product added to product set.


In [32]:
def list_products_in_product_set(
        project_id, location, product_set_id):
    """List all products in a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # List all the products available in the product set.
    products = client.list_products_in_product_set(name=product_set_path)

    # Display the product information.
    for product in products:
        print('Product name: {}'.format(product.name))
        print('Product id: {}'.format(product.name.split('/')[-1]))
        print('Product display name: {}'.format(product.display_name))
        print('Product description: {}'.format(product.description))
        print('Product category: {}'.format(product.product_category))
        print('Product labels: {}'.format(product.product_labels))

list_products_in_product_set('loca-346705', 'us-east1', 'monuments')

Product name: projects/loca-346705/locations/us-east1/products/29th-colored-regiment-monument
Product id: 29th-colored-regiment-monument
Product display name: 29th Colored Regiment Monument
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/loca-346705/locations/us-east1/products/ahavas-sholem-synagogue
Product id: ahavas-sholem-synagogue
Product display name: Ahavas Sholem Synagogue
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/loca-346705/locations/us-east1/products/albertus-magnus-college
Product id: albertus-magnus-college
Product display name: Albertus Magnus College
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/loca-346705/locations/us-east1/products/american-mills-web-shop
Product id: american-mills-web-shop
Product display name: American Mills Web Shop
Product description: 
Product category: general-v1
Product labels: []
Product name: projects/

In [20]:
import re
config = {
    'IMAGE_BUCKET': 'loca_images_db'
}
re.match(
    f'gs://{config["IMAGE_BUCKET"]}' + r"/([^/ .]+)/\d+\.\w+",
    'gs://loca_images_db/whitneyville-congregational-church-hamden-connecticut/0.jpg'
).group(1)

'whitneyville-congregational-church-hamden-connecticut'

In [29]:
import hashlib

hashlib.md5('asdfkljasdlfk'.encode()).hexdigest()

'407114516e897552d266cdc3b4214890'

In [14]:
from google.cloud import vision
from google.cloud.exceptions import Conflict
import io
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/nicholas/programs/yhack/loca-vision/loca-gcloud-key.json'
os.environ['PROJECT_ID'] = 'loca-346705'
os.environ['LOCATION_ID'] = 'us-east4'

def create_reference_image(
        project_id, location, product_id, reference_image_id, gcs_uri):
    """Create a reference image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
        reference_image_id: Id of the reference image.
        gcs_uri: Google Cloud Storage path of the input image.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product.
    product_path = client.product_path(
        project=project_id, location=location, product=product_id)

    # Create a reference image.
    reference_image = vision.ReferenceImage(uri=gcs_uri)

    # The response is the reference image with `name` populated.
    image = client.create_reference_image(
        parent=product_path,
        reference_image=reference_image,
        reference_image_id=reference_image_id)

    # Display the reference image information.
    print('Reference image name: {}'.format(image.name))
    print('Reference image uri: {}'.format(image.uri))


from google.cloud import storage


def list_blobs(bucket_name):
    """Lists all the blobs in the bucket."""
    # bucket_name = "your-bucket-name"

    storage_client = storage.Client()

    # Note: Client.list_blobs requires at least package version 1.17.0.
    return list(storage_client.list_blobs(bucket_name))

# for lipstick_id in ('Lipstick-catepillar.jpg', 'khanacademy.jpg', 'yaleartgallery.jpg'):
#     create_reference_image('loca-346705', 'us-east1', 'lipstick-tank', lipstick_id.split('.')[0], f'gs://loca_images/lipstick-tank/{lipstick_id}')

import re
for blob in list_blobs('loca_images'):
    match = re.match(r'([^/ .]+)/([^/ .]+)\.[png|jpg|jpeg|gif]', blob.name)
    if match:
        product, im = match.groups()
        try:
            create_reference_image('loca-346705', 'us-east1', product, im, f'gs://loca_images/{blob.name}')
        except Conflict as e:
            print('Already exists, skipping')

# for tank_id in ('', 'khanacademy.jpg', 'yaleartgallery.jpg'):
#     create_reference_image('loca-346705', 'us-east1', 'lipstick-tank', lipstick_id.split('.')[0], f'gs://loca_images/lipstick-tank/{lipstick_id}')

Already exists, skipping
Already exists, skipping
Already exists, skipping
Already exists, skipping
Already exists, skipping


In [44]:
def get_similar_products_file(
        project_id,
        location,
        product_set_id,
        product_category,
        file_path,
        filter,
        max_results
):
    """Search similar products to image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_category: Category of the product.
        file_path: Local file path of the image to be searched.
        filter: Condition to be applied on the labels.
                Example for filter: (color = red OR color = blue) AND style = kids
                It will search on all products with the following labels:
                color:red AND style:kids
                color:blue AND style:kids
        max_results: The maximum number of results (matches) to return. If omitted, all results are returned.
    """
    # product_search_client is needed only for its helper methods.
    product_search_client = vision.ProductSearchClient()
    image_annotator_client = vision.ImageAnnotatorClient()

    # Read the image as a stream of bytes.
    with open(file_path, 'rb') as image_file:
        content = image_file.read()

    # Create annotate image request along with product search feature.
    image = vision.Image(content=content)

    # product search specific parameters
    product_set_path = product_search_client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)
    product_search_params = vision.ProductSearchParams(
        product_set=product_set_path,
        product_categories=[product_category],
        filter=filter)
    image_context = vision.ImageContext(
        product_search_params=product_search_params)

    # Search products similar to the image.
    response = image_annotator_client.product_search(
        image,
        image_context=image_context,
        max_results=max_results
    )

    index_time = response.product_search_results.index_time
    print('Product set index time: ', end='')
    print(index_time)

    results = response.product_search_results.results

    print('Search results:')
    for result in results:
        product = result.product

        print('Score(Confidence): {}'.format(result.score))
        print('Image name: {}'.format(result.image))

        print('Product name: {}'.format(product.name))
        print('Product display name: {}'.format(
            product.display_name))
        print('Product description: {}\n'.format(product.description))
        print('Product labels: {}\n'.format(product.product_labels))


# get_similar_products_file('loca-346705', 'us-east1', 'monuments', 'general-v1', 'test_reference/lipstick-tank/yaleartgallery.jpg', None, 3)
get_similar_products_file('loca-346705', 'us-east1', 'monuments', 'general-v1', 'tests/lipstick4geo.jpg', None, 3)

Product set index time: 2022-04-10 00:54:31.090765+00:00
Search results:
Score(Confidence): 0.5112818479537964
Image name: projects/loca-346705/locations/us-east1/products/lipstick-tank/referenceImages/khanacademy
Product name: projects/loca-346705/locations/us-east1/products/lipstick-tank
Product display name: Lipstick (Ascending) on Caterpillar Tracks
Product description: 

Product labels: []

Score(Confidence): 0.254300057888031
Image name: projects/loca-346705/locations/us-east1/products/morse-college/referenceImages/wikipedia
Product name: projects/loca-346705/locations/us-east1/products/morse-college
Product display name: Morse College
Product description: 

Product labels: []



In [41]:
df = pd.read_json('monuments-google.json')
# df['coord'] = [f'POINT({x["lat"]:.5f} {x["lon"]:.5f})' for x in df['coord']]
df

,name,desc,coord,image_urls
0,William Andrew House,"The William Andrew House, also known as the Ri...","{'lat': 41.26527778, 'lon': -73.01222222}",[gs://loca_images_db/william-andrew-house/0.pn...
1,"West Haven, Connecticut","West Haven is a city in New Haven County, Conn...","{'lat': 41.27388889, 'lon': -72.96777778}",[gs://loca_images_db/west-haven-connecticut/0....
2,West Haven station,West Haven station is a commuter rail station ...,"{'lat': 41.271142, 'lon': -72.963199}",[gs://loca_images_db/west-haven-station/0.jpg]
3,West Haven High School,West Haven High School is a secondary school l...,"{'lat': 41.26, 'lon': -72.9585}",[]
4,Savin Rock,"Savin Rock is a section of West Haven, Connect...","{'lat': 41.25972222, 'lon': -72.94527778}",[gs://loca_images_db/savin-rock/0.jpg]
...,...,...,...,...
224,Eli Whitney Museum,"The Eli Whitney Museum, in Hamden, Connecticut...","{'lat': 41.33583333, 'lon': -72.91055556}","[gs://loca_images_db/eli-whitney-museum/0.jpg,..."
225,"Whitneyville Congregational Church (Hamden, Co...","The Whitneyville Congregational Church, now th...","{'lat': 41.34666667, 'lon': -72.91305556}",[gs://loca_images_db/whitneyville-congregation...
226,New Haven Country Club,New Haven Country Club is a private country cl...,"{'lat': 41.349043, 'lon': -72.907231}",[]
227,Pistol Factory Dwelling,The Pistol Factory Dwelling is a historic hous...,"{'lat': 41.34916667, 'lon': -72.91416667}",[gs://loca_images_db/pistol-factory-dwelling/0...


In [74]:
from google.cloud import vision
import io
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/nicholas/programs/yhack/loca-vision/loca-gcloud-key.json'
os.environ['PROJECT_ID'] = 'loca-346705'
os.environ['LOCATION_ID'] = 'us-east1'

from google.cloud import vision

def create_reference_image(
        project_id, location, product_id, reference_image_id, gcs_uri):
    """Create a reference image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
        reference_image_id: Id of the reference image.
        gcs_uri: Google Cloud Storage path of the input image.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product.
    product_path = client.product_path(
        project=project_id, location=location, product=product_id)

    # Create a reference image.
    reference_image = vision.ReferenceImage(uri=gcs_uri)

    # The response is the reference image with `name` populated.
    image = client.create_reference_image(
        parent=product_path,
        reference_image=reference_image,
        reference_image_id=reference_image_id)

    # Display the reference image information.
    print('Reference image name: {}'.format(image.name))
    print('Reference image uri: {}'.format(image.uri))


def web_entities_include_geo_results(path):
    """Detects web annotations given an image, using the geotag metadata
    in the image to detect web entities."""

    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    web_detection_params = vision.WebDetectionParams(
        include_geo_results=True)
    image_context = vision.ImageContext(
        web_detection_params=web_detection_params)

    response = client.web_detection(image=image, image_context=image_context)

    for entity in response.web_detection.web_entities:
        print('\n\tScore      : {}'.format(entity.score))
        print(u'\tDescription: {}'.format(entity.description))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

web_entities_include_geo_results('tests/lipstick1.jpg')


	Score      : 0.4916285574436188
	Description: Façade

	Score      : 0.4340176284313202
	Description: Roof

	Score      : 0.40162393450737
	Description: Wall


In [78]:
def detect_web(path):
    """Detects web annotations given an image."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = client.web_detection(image=image)
    annotations = response.web_detection

    if annotations.best_guess_labels:
        for label in annotations.best_guess_labels:
            print('\nBest guess label: {}'.format(label.label))

    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images found:'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print('\n\tPage url   : {}'.format(page.url))

            if page.full_matching_images:
                print('\t{} Full Matches found: '.format(
                       len(page.full_matching_images)))

                for image in page.full_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

            if page.partial_matching_images:
                print('\t{} Partial Matches found: '.format(
                       len(page.partial_matching_images)))

                for image in page.partial_matching_images:
                    print('\t\tImage url  : {}'.format(image.url))

    if annotations.web_entities:
        print('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('\n\tScore      : {}'.format(entity.score))
            print(u'\tDescription: {}'.format(entity.description))

    if annotations.visually_similar_images:
        print('\n{} visually similar images found:\n'.format(
            len(annotations.visually_similar_images)))

        for image in annotations.visually_similar_images:
            print('\tImage url    : {}'.format(image.url))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

detect_web('tests/lipstick5.jpg')


Best guess label: architecture

7 Web entities found: 

	Score      : 0.5086863040924072
	Description: Architecture

	Score      : 0.3930414319038391
	Description: Angle

	Score      : 0.37575626373291016
	Description: Façade

	Score      : 0.3674440383911133
	Description: Roof

	Score      : 0.322299987077713
	Description: LON:0MWC

	Score      : 0.0
	Description: Geometry

	Score      : 0.0
	Description: Mathematics

10 visually similar images found:

	Image url    : https://i.pinimg.com/736x/1b/4b/07/1b4b0748b00bd1043e1e19c9934d5eb8.jpg
	Image url    : https://cdn03.plentymarkets.com/csse1tl3e797/item/images/12650/full/profhome-tapte-wallpaper-intl-958331-8.jpg
	Image url    : https://ctl.s6img.com/society6/img/GrYHVQIpfPC4RhFXqX-J5K5DH-s/h_264,w_264/wall-murals/8x8/lifestyle/~artwork/s6-original-art-uploads/society6/uploads/misc/67893f363ad24cde81cf52046870f633/~~/african-american-masterpiece-the-thankful-poor-by-henry-ossawa-tanner-wall-murals.jpg
	Image url    : https://cdn.shop

In [73]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations

    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices:
            print(' - ({}, {})'.format(vertex.x, vertex.y))

localize_objects('tests/lipstick1.jpg')

Number of objects found: 1

Packaged goods (confidence: 0.6741136908531189)
Normalized bounding polygon vertices: 
 - (0.12483152002096176, 0.6266592144966125)
 - (0.4863053262233734, 0.6266592144966125)
 - (0.4863053262233734, 0.8532521724700928)
 - (0.12483152002096176, 0.8532521724700928)
